In [1]:
import pandas as pd
pd.set_option("display.max_columns", 50)
import warnings
warnings.filterwarnings('ignore')

path1 = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"
path2 = "https://raw.githubusercontent.com/Soyoung-Yoon/data_01/main/"


**8-1) 레스토랑의 팁(tip)분석**

다음 조건을 만족하는 분석을 수행하시오.
- 요일(day)별 total_bill의 평균이 전체 total_bill 평균보다 높은 요일의 데이터를 추출하시오.
- 위에서 추출된 데이터를 사용하여, day와 smoker로 그룹화하여 tip의 평균을 구하고, 가장 평균 tip이 높은 (day, smoker) 조합을 찾으시오.
- 위의 가장 평균 tip이 높은 조합의 데이터에 대해, total_bill의 표준편차를 구하시오.
- 표준편차 값을 반올림하여 소수점 아래 3 자리까지 출력하시오.

In [ ]:
df = pd.read_csv(path1 + "tips.csv")
# print(df.head(3), df.shape) # (244, 7)

# 요일(day)별 total_bill의 평균이 전체 total_bill 평균보다 높은 요일의 데이터를 추출하시오.
s = df.groupby('day')['total_bill'].transform('mean')
df = df[df['total_bill'].mean() < s]

# 추출된 데이터를 사용하여, day와 smoker로 그룹화하여 tip의 평균을 구하고,
# 가장 평균 tip이 높은 (day, smoker) 조합을 찾으시오.
mean_tip = df.groupby(['day','smoker'],as_index=False)['tip'].mean().sort_values('tip', ascending=False).max()
# print(f"day: {mean_tip['day']}, smoker: {mean_tip['smoker']}") # mean_tip['smoker']

# 위의 가장 평균 tip이 높은 조합의 데이터에 대해, total_bill의 표준편차를 구하시오.
result = round(df[(df['day'] == mean_tip['day']) & (df['smoker'] == mean_tip['smoker'])]['total_bill'].std(ddof=1), 3)
print(result) # 10.443


10.443


**8-2a) 프로모션 기획**

온라인 플랫폼에서는 특정 요일에 이벤트의 발생 빈도를 활용한 프로모션을 기획하려 한다.

다음 절차에 따라 문제를 해결하시오.
- 시계열 데이터를 이용해 각 이벤트가 발생한 '요일'을 파생 변수로 추가한 뒤, 요일별 value 총합을 구하시오.
- 위의 데이터를 사용하여, 가장 큰 3개 값의 평균을 구해 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3))

# 시계열 데이터를 이용해 각 이벤트가 발생한 '요일'을 파생 변수로 추가한 뒤, 요일별 value 총합을 구하시오.
df['date'] = df['date'].astype('datetime64[ns]')
df['요일'] = df['date'].dt.day_name('ko_KR')
daysum = df.groupby('요일', as_index=False)['value'].sum()

# 위의 데이터를 사용하여, 가장 큰 3개 값의 평균을 구해 반올림하여 정수로 출력
result = round(daysum.sort_values('value', ascending=False).head(3)['value'].mean())
print(result) # 8342

8342


**8-2b) 프로모션 기획**

온라인 플랫폼에서는 특정 요일에 이벤트의 발생 빈도를 활용한 프로모션을 기획하려 한다.

다음 절차에 따라 문제를 해결하시오.
- 각 이벤트가 발생한 날짜에서 '요일' 정보를 추출한다.
- 요일별로 value의 총합을 구하여, 가장 높은 합계를 가진 요일 3개를 식별한다.
- 이 3개 요일에 해당하는 모든 이벤트의 value 평균을 계산하고, 그 값을 반올림하여 정수로 출력하시오.


In [42]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3))

# 각 이벤트가 발생한 날짜에서 '요일' 정보를 추출
df['요일'] = df['date'].astype('datetime64[ns]').dt.day_name('ko_KR')
# print(df.head(3))

# 요일별로 value의 총합을 구하여, 가장 높은 합계를 가진 요일 3개를 식별
daysum = df.groupby('요일')['value'].sum().reset_index()
s = daysum.sort_values('value', ascending=False).head(3)['요일']

# 이 3개 요일에 해당하는 모든 이벤트의 value 평균을 계산하고, 그 값을 반올림하여 정수로 출력
result = round(df[df['요일'].isin(s)]['value'].mean())
print(result) # 57

57


**8-3) 이벤트별 월간 성과**

다양한 이벤트 카테고리별 월간 성과 분석을 위한 과제이다.

다음 절차에 따라 문제를 해결하시오.
- **주어진 데이터**에 대해 카테고리(category2), 연도(year), 월(month) 단위로 value의 총합을 집계하시오.
- 이후, (category2, year)별로 집계된 값 중 가장 value 합계가 높은 월을 선택하시오.
- 이렇게 선택된 월별 최댓값들을 모두 더한 뒤, 그 합계를 반올림하여 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_05.csv")
# print(df.head(3))

# 주어진 데이터에 대해 카테고리(category2), 연도(year), 월(month) 단위로 value의 총합을 집계
df['date'] = df['date'].astype('datetime64[ns]')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df2 = df.groupby(['category2','year','month'])['value'].sum().reset_index()
# print(df2.head(3))

#  (category2, year)별로 집계된 값 중 가장 value 합계가 높은 월을 선택
df3 = df2.loc[df2.groupby(['category2','year'])['value'].idxmax()].reset_index(drop=True)
# print(df3.head(3))

# 이렇게 선택된 월별 최댓값들을 모두 더한 뒤, 그 합계를 반올림하여 정수로 출력
result = round(df3['value'].sum())
print(result) # 7605

7605


**8-4) 이상 감지 및 성능 분석**

시스템 비활성 이벤트의 이상 감지 및 성능 분석을 위한 과제이다.

다음 절차에 따라 문제를 해결하시오.
- 2020년 1월부터 6월까지의 데이터 중 event가 'Idle'인 이벤트만 필터링하시오.
- 월(month)별로 해당 이벤트들의 value 중앙값을 구하시오.
- 해당 월의 value 중앙값을 기준으로 그 달에 발생한 이벤트 중 value가 중앙값 이하인 이벤트만 선택하시오.
- 이렇게 선택된 이벤트들의 총 개수를 정수로 출력하시오.

In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3), df.shape) # (1000, 4)

# 2020년 1월부터 6월까지의 데이터 중 event가 'Idle'인 이벤트만 필터링
df['date'] = df['date'].astype('datetime64[ns]')
cond1 = (df['date'].dt.year==2020) & (df['date'].dt.month >= 1) & (df['date'].dt.month <= 6)
df = df[cond1]
df = df[df['event'] == 'Idle']
# print(df.shape) # (30, 4)

# 월(month)별로 해당 이벤트들의 value 중앙값을 구하시오.
df['month'] = df['date'].dt.month
median = df.groupby('month')['value'].transform('median')
# print(median)

# 해당 월의 value 중앙값을 기준으로 그 달에 발생한 이벤트 중 value가 중앙값 이하인 이벤트만 선택하시오.
# 이렇게 선택된 이벤트들의 총 개수를 정수로 출력하시오.
result = df[df['value'] <= median]['event'].count()
print(int(result)) # 16

16


**8-5) 야간 시간대 이벤트**

다음은 야간 시간대에 발생한 이벤트 유형 분석을 통해 사용자 활동 패턴을 파악하고자 하는 과제이다.

다음 절차에 따라 문제를 해결하시오.
- 먼저, 시간(hour) 정보가 22시부터 02시까지(22:00:00 ~ 02:59:59)인 데이터를 야간 이벤트로 간주한다.
- 이 야간 이벤트를 기준으로, (month, event)별 value 평균값을 구하시오.
- 각 월(month)에서 value 평균이 가장 높은 이벤트 유형을 하나씩 선택하고, 그 중 'Start'와 'Stop'이벤트의 빈도 수 합을 정수로 출력하시오.


In [ ]:
df = pd.read_csv(path2 + "event_log_04.csv")
# print(df.head(3))

# 시간(hour) 정보가 22시부터 02시까지(22:00:00 ~ 02:59:59)인 데이터를 야간 이벤트로 간주
from datetime import time
df['time'] = pd.to_datetime(df['time']).dt.time
df = df[(df['time'] >= time(22, 0, 0)) | (df['time'] <= time(2, 59, 59))]
# print(df)

# 이 야간 이벤트를 기준으로, (month, event)별 value 평균값을 구하시오.
df['date'] = df['date'].astype('datetime64[ns]')
df['month'] = df['date'].dt.month
df2 = df.groupby(['month','event'])['value'].mean().reset_index()

# 각 월(month)에서 value 평균이 가장 높은 이벤트 유형을 하나씩 선택하고, 
# 그 중 'Start'와 'Stop'이벤트의 빈도 수 합을 정수로 출력
df3 = df2.loc[df2.groupby('month')['value'].idxmax()].reset_index(drop=True)
result = int(df3['event'].str.contains('Start|Stop').sum())
print(result) # 7

7
